<a href="https://colab.research.google.com/github/mottasilvia/UCU-NLP/blob/main/PLN_Actividad_1_RedesNeuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential # Import Sequential from tensorflow.keras
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN # Import recurrent layers from tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout # Import core layers from tensorflow.keras
from tensorflow.keras.layers import Embedding # Import Embedding layer from tensorflow.keras
from tensorflow.keras.layers import BatchNormalization # Import BatchNormalization from tensorflow.keras
from tensorflow.keras.utils import to_categorical # Import to_categorical for one-hot encoding
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D # Import additional layers from tensorflow.keras
from tensorflow.keras.preprocessing import sequence, text # Import preprocessing modules from tensorflow.keras
from tensorflow.keras.callbacks import EarlyStopping # Import EarlyStopping from tensorflow.keras


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.metrics import classification_report

In [ ]:
# Verificar la disponibilidad de la GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


Num GPUs Available:  1
Default GPU Device: /device:GPU:0


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
carpeta_laboratorios = "/"
path_laboratorios = '/content/drive/My Drive/'
#df = pd.read_csv(path_laboratorios+"Canciones_limpias_finalisismo.csv")

#train = pd.read_csv(path_laboratorios+"train.csv")
#test = pd.read_csv(path_laboratorios+"test.csv")
#validation = pd.read_csv(path_laboratorios+"val.csv")


Mounted at /content/drive


In [ ]:
import zipfile
import os

def unzip_to_folder(zip_files, target_dir="unzipped_files"):
  os.makedirs(target_dir, exist_ok=True)

  for zip_file in zip_files:
    filename, _ = os.path.splitext(os.path.basename(zip_file))
    folder_name = target_dir
    os.makedirs(folder_name, exist_ok=True)

    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
      zip_ref.extractall(folder_name)

if __name__ == "__main__":
  # Example usage
  zip_files = ["/content/drive/MyDrive/UCU-TesisFinal/aarchive (3).zip"]
  unzip_to_folder(zip_files, "/content/Movie")


In [ ]:
from google.colab import drive
import pandas as pd
import os


# Define los paths a los directorios que contienen las reseñas
neg_dir = '/content/Movie/txt_sentoken/neg'
pos_dir = '/content/Movie/txt_sentoken/pos'

# Función para leer reseñas de un directorio y asignar una etiqueta
def load_reviews_from_directory(directory, tag):
    reviews = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                review = file.read()
                reviews.append((review, tag))
    return reviews

# Verificar si los directorios existen
if os.path.exists(neg_dir) and os.path.exists(pos_dir):
    # Cargar reseñas negativas y positivas
    negative_reviews = load_reviews_from_directory(neg_dir, 'negative')
    positive_reviews = load_reviews_from_directory(pos_dir, 'positive')

    # Combinar las reseñas en un único DataFrame
    all_reviews = negative_reviews + positive_reviews
    df = pd.DataFrame(all_reviews, columns=['review', 'tag'])

    # Mostrar el DataFrame
    from IPython.display import display
    display(df)

    # Verificar las primeras filas del DataFrame
    print(df.head())
else:
    print("Los directorios especificados no existen. Verifica los paths y vuelve a intentarlo.")


,review,tag
0,i have nothing against unabashedly romantic fi...,negative
1,"in 1989 , director edward zwick began his care...",negative
2,"around the end of 1998 , a japanese cartoon ca...",negative
3,the main problem with martin lawrence's pet pr...,negative
4,what would inspire someone who cannot write or...,negative
...,...,...
1995,it was a rainy friday afternoon in columbus wh...,positive
1996,dreamworks pictures presents a jinks/ cohen co...,positive
1997,plot : derek zoolander is a male model . \nhe ...,positive
1998,"susan granger's review of "" osmosis jones "" ( ...",positive


                                              review       tag
0  i have nothing against unabashedly romantic fi...  negative
1  in 1989 , director edward zwick began his care...  negative
2  around the end of 1998 , a japanese cartoon ca...  negative
3  the main problem with martin lawrence's pet pr...  negative
4  what would inspire someone who cannot write or...  negative


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el DataFrame en train (60%) y temp (40%)
train_df, temp_df = train_test_split(df, test_size=0.4, random_state=42, stratify=df['tag'])

# Dividir el temp_df en validation (50% de 40% -> 20%) y test (50% de 40% -> 20%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['tag'])

# Mostrar los tamaños de los conjuntos de datos
print(f"Tamaño del conjunto de entrenamiento: {len(train_df)}")
print(f"Tamaño del conjunto de validación: {len(val_df)}")
print(f"Tamaño del conjunto de prueba: {len(test_df)}")

# Mostrar las primeras filas de cada conjunto de datos para verificación
print("Conjunto de entrenamiento:")
print(train_df.head())

print("\nConjunto de validación:")
print(val_df.head())

print("\nConjunto de prueba:")
print(test_df.head())


Tamaño del conjunto de entrenamiento: 1200
Tamaño del conjunto de validación: 400
Tamaño del conjunto de prueba: 400
Conjunto de entrenamiento:
                                                 review       tag
1523  many people dislike french films for their lac...  positive
1202  ingredients : james bond , scuba scene , car c...  positive
468   i have a confession . \neven though i am a mov...  negative
578   i didn't come into city of angels expecting gr...  negative
1029  the relaxed dude rides a roller coaster \nthe ...  positive

Conjunto de validación:
                                                 review       tag
1152  meet joe black ( reviewed on nov . 27/98 ) \ns...  positive
1762  there must be some unwritten rule that states ...  positive
1142  there's a moment in schindler's list when a nu...  positive
612   nearly every film tim burton has directed has ...  negative
202   retrospective : city of the living dead ( 1980...  negative

Conjunto de prueba:
                  

# **Redes Neuronales Recurrentes**

Veamos el máximo de palabras por review para tener una idea luego a la hora de paddear y todas esas cosas.-

In [ ]:
max_words_per_review = df['review'].apply(lambda x: len(str(x).split())).max()
max_words_per_review

2678

In [ ]:
max_len = 2700 #Si bien el máximo es 2678, ponemos 2700 para tener un poco más de holgura. Se va a usar después para padear!

In [ ]:
xtrain = train_df.review.values
xvalid = val_df.review.values
xtest = test_df.review.values
ytrain = train_df.tag.values
yvalid = val_df.tag.values
ytest = test_df.tag.values



In [ ]:
xtrain


array(['every once in a while a movie comes along that completely redefines the genre : with dramas , it was citizen kane , with arthouse it was pulp fiction , and with comedy it was , well , that jim carrey guy ( okay , so he\'s not a movie , but he did have a huge influence on the genre . \nnot to mention an expensive one . ) \nsometimes a movie even combines them all into a big , sprawling motion picture event , as did forrest gump four years ago . \nwith action films , it was aliens , whic was released to much hype seven years after it\'s equally-innovative parent , alien ( 1979 ) . \ndirected and written by james cameron ( t2 : judgement day , the abyss , true lies ) , the authority on action films , it was a masterful encore to his sci-fi thriller the terminator ( 1984 ) . \nwhile the original alien film was a dark , enclosed horror film that featured one alien slowly massacering a horrified crew , james cameron took the big-budget action film with aliens , which featured multipl

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# Usando el tokenizer de Keras
tokenizer = Tokenizer(num_words=None)  # Puedes ajustar num_words si deseas limitar el tamaño del vocabulario

# Ajustar el tokenizer a los textos de entrenamiento y validación
tokenizer.fit_on_texts(list(xtrain) + list(xvalid))

# Convertir los textos a secuencias de enteros
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xvalid_seq = tokenizer.texts_to_sequences(xvalid)
xtest_seq = tokenizer.texts_to_sequences(xtest)

# Aplicar padding a las secuencias
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len, padding='post')
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len, padding='post')
xtest_pad = pad_sequences(xtest_seq, maxlen=max_len, padding='post')

# Obtener el índice de palabras del tokenizer
word_index = tokenizer.word_index

# Verificar los resultados
print(f"Tokenizador ajustado con {len(word_index)} palabras únicas.")
print(f"Ejemplo de secuencia paddeada (xtrain_pad[0]): {xtrain_pad[0]}")
print(f"Forma de xtrain_pad: {xtrain_pad.shape}")
print(f"Forma de xvalid_pad: {xvalid_pad.shape}")
print(f"Forma de xtest_pad: {xtest_pad.shape}")


Tokenizador ajustado con 39609 palabras únicas.
Ejemplo de secuencia paddeada (xtrain_pad[0]): [ 112   99 7160 ...    0    0    0]
Forma de xtrain_pad: (1200, 2700)
Forma de xvalid_pad: (400, 2700)
Forma de xtest_pad: (400, 2700)


In [ ]:
xtrain_pad

array([[ 112,   99, 7160, ...,    0,    0,    0],
       [3862,  328,  942, ...,    0,    0,    0],
       [  17,   30,    2, ...,    0,    0,    0],
       ...,
       [4326,    6,   76, ...,    0,    0,    0],
       [  11,    2,  448, ...,    0,    0,    0],
       [  17,  123,  120, ...,    0,    0,    0]], dtype=int32)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import numpy as np

# Convertir etiquetas de texto a enteros
label_encoder = LabelEncoder()
ytrain_encoded = label_encoder.fit_transform(ytrain)
yvalid_encoded = label_encoder.transform(yvalid)
ytest_encoded = label_encoder.transform(ytest)

# Convertir los enteros a representación one-hot
ytrain_one_hot = to_categorical(ytrain_encoded)
yvalid_one_hot = to_categorical(yvalid_encoded)
ytest_one_hot = to_categorical(ytest_encoded)

# Verificar las formas de los arrays one-hot
print(f"Forma de ytrain_one_hot: {ytrain_one_hot.shape}")
print(f"Forma de yvalid_one_hot: {yvalid_one_hot.shape}")
print(f"Forma de ytest_one_hot: {ytest_one_hot.shape}")

# Opcional: Verificar los primeros ejemplos de etiquetas one-hot
print("Ejemplo de etiquetas one-hot (ytrain_one_hot[0]):")
print(ytrain_one_hot[0])


Forma de ytrain_one_hot: (1200, 2)
Forma de yvalid_one_hot: (400, 2)
Forma de ytest_one_hot: (400, 2)
Ejemplo de etiquetas one-hot (ytrain_one_hot[0]):
[0. 1.]


En una RNN ingresamos una review palabra por palabra. Representamos cada palabra como un vector utilizando la técnica del one hot encoding. El vector va a tener cantidad de palabras en el vocabulario + 1 domensiones.

Lo que hace keras Tokenizer es, toma todas las palabras únicas en el corpus, forma un diccionario con palabras como claves y su número de ocurrencias como valores (es el word_index), luego ordena el diccionario en orden descendente de conteos.

Luego asigna a la primer palabra el valor 1, a la segunda el valor 2 y así sucesivamente.

Así que supongamos que la palabra 'que' es la que se repite más en todas las canciones, a esa se le asignará el índice 1 y el vector que representa la palabra 'que' sería un vector one-hot con valor 1 en la posición 1 y resto ceros.

La primera línea del modelo "Sequential()" le dice a Keras que construiremos nuestra red secuencialmente.

Luego, primero agregamos la capa de incrustación. La capa de Embedding que es también una capa de neuronas que toma como entrada el vector one-hot n-ésimo de cada palabra y lo convierte en un vector de 300 dimensiones, nos da el enbeddubg de palabras similar a word2vec. Podríamos haber utilizado word2vec, pero la capa de Embedding aprende durante el entrenamiento para mejorar la forma de embeddear.

A continuación, agregamos 100 unidades LSTM sin ningún dropout ni regularización.

Por último, agregamos 5 neuronas (ya que tenemos 5 posibles clases) con función sigmoidea que toma la salida de las 100 células LSTM (tener en cuenta que tenemos 100 células LSTM, no capas) para predecir los resultados y luego compilamos el modelo usando adam optimizer


# RNNs

In [ ]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,#Cantidad de palabras distintas del corpus
                     300,#tamaño del vector con el que va a encodear cada palabra (del 1-hot lo pasa a un vector de 300 dimensiones)
                     input_length=max_len)) #El máximo de palabras que puede tener una canción.
    model.add(SimpleRNN(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2700, 300)         11829000  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_3 (Dense)             (None, 50)                5050      
                                                                 
 dense_4 (Dense)             (None, 20)                1020      
                                                                 
 dense_5 (Dense)             (None, 2)                 42        
                                                                 
Total params: 11875212 (45.30 MB)
Trainable params: 11875212 (45.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: u

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report

# Entrenar el modelo
model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=64, validation_data=(xvalid_pad, yvalid_one_hot))

# Predecir en el conjunto de validación
preds = np.argmax(model.predict(xvalid_pad), axis=-1)

# Calcular la precisión
accuracy = metrics.accuracy_score(yvalid_encoded, preds)
print(f"Accuracy: {accuracy}")

# Generar el reporte de clasificación
target_names = label_encoder.classes_  # Esto debería ser ['negative', 'positive']
clas_report = classification_report(yvalid_encoded, preds, target_names=target_names)
print(clas_report)


Epoch 1/20
19/19 [==============================] - 111s 6s/step - loss: 0.7034 - accuracy: 0.4900 - val_loss: 0.6932 - val_accuracy: 0.5400
Epoch 2/20
19/19 [==============================] - 96s 5s/step - loss: 0.7021 - accuracy: 0.4858 - val_loss: 0.6980 - val_accuracy: 0.5000
Epoch 3/20
19/19 [==============================] - 122s 6s/step - loss: 0.6953 - accuracy: 0.5292 - val_loss: 0.7042 - val_accuracy: 0.5075
Epoch 4/20
19/19 [==============================] - 103s 5s/step - loss: 0.7001 - accuracy: 0.4942 - val_loss: 0.7030 - val_accuracy: 0.4725
Epoch 5/20
19/19 [==============================] - 104s 6s/step - loss: 0.6977 - accuracy: 0.4858 - val_loss: 0.7011 - val_accuracy: 0.5050
Epoch 6/20
19/19 [==============================] - 104s 5s/step - loss: 0.7002 - accuracy: 0.4908 - val_loss: 0.6988 - val_accuracy: 0.4700
Epoch 7/20
19/19 [==============================] - 107s 6s/step - loss: 0.7018 - accuracy: 0.4858 - val_loss: 0.6940 - val_accuracy: 0.5125
Epoch 8/20
19/

# Convolutional

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn import metrics
from sklearn.metrics import classification_report

# Crear el modelo CNN
model = Sequential()
model.add(Embedding(len(word_index) + 1, 300, input_length=max_len))
model.add(Conv1D(128, 5, activation="relu"))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation="relu"))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation="relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Ajustar el número de unidades a 2

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

# Entrenar el modelo
model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=64, validation_data=(xvalid_pad, yvalid_one_hot))

# Predecir en el conjunto de validación
preds = np.argmax(model.predict(xvalid_pad), axis=-1)

# Calcular la precisión
accuracy = metrics.accuracy_score(yvalid_encoded, preds)
print(f"Accuracy: {accuracy}")

# Generar el reporte de clasificación
target_names = label_encoder.classes_  # Esto debería ser ['negative', 'positive']
clas_report = classification_report(yvalid_encoded, preds, target_names=target_names)
print(clas_report)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 2700, 300)         11829000  
                                                                 
 conv1d (Conv1D)             (None, 2696, 128)         192128    
                                                                 
 max_pooling1d (MaxPooling1  (None, 539, 128)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 535, 128)          82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 107, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 103, 128)         

In [ ]:
!wget -O /content/drive/MyDrive/pretrainedwordvectorsforspanish/cc.es.300.vec.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz


--2024-07-02 22:44:54--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.156.133.37, 108.156.133.4, 108.156.133.117, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.156.133.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1285580896 (1.2G) [binary/octet-stream]
Saving to: ‘/content/drive/MyDrive/pretrainedwordvectorsforspanish/cc.es.300.vec.gz’

/content/drive/MyDr 100%[===================>]   1.20G  23.7MB/s    in 53s     

2024-07-02 22:45:48 (23.0 MB/s) - ‘/content/drive/MyDrive/pretrainedwordvectorsforspanish/cc.es.300.vec.gz’ saved [1285580896/1285580896]



In [ ]:
!gunzip /content/drive/MyDrive/pretrainedwordvectorsforspanish/cc.es.300.vec.gz


gzip: /content/drive/MyDrive/pretrainedwordvectorsforspanish/cc.es.300.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [ ]:
import os
from gensim.models.keyedvectors import KeyedVectors

# Verificar si el archivo existe en la ruta especificada
wordvectors_file_vec = '/content/drive/MyDrive/pretrainedwordvectorsforspanish/cc.es.300.vec'
if os.path.exists(wordvectors_file_vec):
    print("El archivo se ha encontrado correctamente.")
    # Cargar los embeddings preentrenados
    wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, binary=False)
    print("Embeddings de FastText cargados exitosamente.")
else:
    print("El archivo no se encuentra en la ruta especificada.")


El archivo se ha encontrado correctamente.
Embeddings de FastText cargados exitosamente.


# **Word2Vec preentrenado**

In [ ]:
# Crear una matriz de embeddings donde cada fila representa un vector de una palabra
embedding_dim = 300
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in wordvectors:
        embedding_matrix[i] = wordvectors[word]
    else:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embedding_dim)

# Crear un DataFrame para visualizar la matriz de embeddings
# Seleccionar las primeras 10 filas para visualizar
embedding_df = pd.DataFrame(embedding_matrix[:10], columns=[f"dim_{i}" for i in range(embedding_dim)])
print(embedding_df)


    dim_0   dim_1   dim_2   dim_3   dim_4   dim_5   dim_6   dim_7   dim_8  \
0  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   
1  0.0844  0.1832  0.1464  0.0012  0.0093  0.0685  0.0003  0.0620  0.0818   
2  0.0409 -0.0381  0.1203  0.0221  0.0250  0.0294  0.0568  0.0220 -0.0372   
3 -0.0496  0.0836 -0.0782 -0.0708 -0.0174  0.0452 -0.0902  0.0514  0.0917   
4 -0.0828  0.3689  0.2216 -0.1033  0.0435  0.1911  0.1409 -0.0305  0.1279   
5 -0.3831  0.2311 -0.0661 -0.0249 -0.1404  0.1883  0.1389  0.1286  0.0913   
6  0.0493  0.2054 -0.5362 -0.1148 -0.0362  0.1562  0.1540  0.0439  0.0193   
7 -0.2347  0.2863  0.1340  0.0471  0.0360  0.4923 -0.3978  0.0184  0.0242   
8  0.1588  0.0520  0.0253 -0.0447 -0.0017  0.0442  0.0296  0.0692  0.0442   
9  0.2019 -0.0950 -0.1225 -0.0689  0.1043  0.3920 -0.2888  0.4975 -0.2205   

    dim_9  ...  dim_290  dim_291  dim_292  dim_293  dim_294  dim_295  dim_296  \
0  0.0000  ...   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   

In [ ]:
def representar_review_como_vector(review):
    palabras = review.split()
    vectores = np.zeros(300)  # Suponiendo que los embeddings tienen 300 dimensiones
    palabras_encontradas = 0
    for palabra in palabras:
        try:
            vector = wordvectors.word_vec(palabra)
            vectores = vectores + vector
            palabras_encontradas = palabras_encontradas + 1
        except KeyError:
            print("No está la palabra " + palabra)

    if palabras_encontradas > 0:
        promedio = vectores / palabras_encontradas
    else:
        promedio = vectores
    return vectores, promedio



In [ ]:
# Seleccionar una review específica para probar
review_ejemplo = train_df[train_df['review'].str.contains('the only problem')]['review'].values[0]

# Obtener los vectores suma y promedio de la review
suma, promedio = representar_review_como_vector(review_ejemplo)


No está la palabra linklater's
No está la palabra $23
No está la palabra immerses
No está la palabra twentysomething
No está la palabra college-town
No está la palabra spouting
No está la palabra inadequacy
No está la palabra pervasiveness
No está la palabra they've
No está la palabra they'll
No está la palabra linklater
No está la palabra it's
No está la palabra linklater
No está la palabra meandering
No está la palabra pseudo-intellectuals
No está la palabra bribery-based
No está la palabra scooby-doo
No está la palabra unhinged
No está la palabra conspiracies
No está la palabra linklater
No está la palabra what's
No está la palabra aren't
No está la palabra strangeness
No está la palabra linklater
No está la palabra oddballs
No está la palabra well-deserved
No está la palabra can't
No está la palabra linklater


<ipython-input-21-907522245eb5>:7: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  vector = wordvectors.word_vec(palabra)


In [ ]:
# Encontrar las palabras más similares al vector promedio
print("Palabras similares al vector promedio:")
print(wordvectors.similar_by_vector(promedio, topn=10))

# Encontrar las palabras más similares al vector suma
print("Palabras similares al vector suma:")
print(wordvectors.similar_by_vector(suma, topn=10))

# Normalizar el vector suma y encontrar palabras similares
normalized_v = suma / np.linalg.norm(suma)
print("Palabras similares al vector suma normalizado:")
print(wordvectors.similar_by_vector(normalized_v, topn=10))


Palabras similares al vector promedio:
[('of', 0.7201033234596252), ('meI', 0.7176797389984131), ('the', 0.7163772583007812), ('that', 0.7091971039772034), ('to', 0.705650269985199), ('and', 0.7054423689842224), ('is', 0.6933733224868774), ('day.', 0.6874086260795593), ('it', 0.6863765716552734), ('whith', 0.6731469035148621)]
Palabras similares al vector suma:
[('of', 0.72010338306427), ('meI', 0.7176797986030579), ('the', 0.7163771986961365), ('that', 0.7091971039772034), ('to', 0.705650269985199), ('and', 0.7054423689842224), ('is', 0.6933733224868774), ('day.', 0.6874086260795593), ('it', 0.6863766312599182), ('whith', 0.6731469035148621)]
Palabras similares al vector suma normalizado:
[('of', 0.72010338306427), ('meI', 0.7176797389984131), ('the', 0.7163771986961365), ('that', 0.7091971039772034), ('to', 0.705650269985199), ('and', 0.7054423093795776), ('is', 0.6933733820915222), ('day.', 0.6874086260795593), ('it', 0.6863765716552734), ('whith', 0.6731469035148621)]


## **Implementamos Doc2Vec**

In [ ]:

problem = train_df[train_df['review'].str.contains('the only problem')]['review'].values[0]
suma, promedio = representar_review_como_vector(problem)
print(suma)
print(wordvectors.similar_by_vector(promedio, topn=10))
print(promedio)
print(wordvectors.similar_by_vector(suma, topn=10))
normalized_v = suma/np.linalg.norm(suma)
print(wordvectors.similar_by_vector(normalized_v, topn=10))

<ipython-input-21-907522245eb5>:7: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  vector = wordvectors.word_vec(palabra)


No está la palabra linklater's
No está la palabra $23
No está la palabra immerses
No está la palabra twentysomething
No está la palabra college-town
No está la palabra spouting
No está la palabra inadequacy
No está la palabra pervasiveness
No está la palabra they've
No está la palabra they'll
No está la palabra linklater
No está la palabra it's
No está la palabra linklater
No está la palabra meandering
No está la palabra pseudo-intellectuals
No está la palabra bribery-based
No está la palabra scooby-doo
No está la palabra unhinged
No está la palabra conspiracies
No está la palabra linklater
No está la palabra what's
No está la palabra aren't
No está la palabra strangeness
No está la palabra linklater
No está la palabra oddballs
No está la palabra well-deserved
No está la palabra can't
No está la palabra linklater
[-7.20710010e+00  3.93489001e+01 -7.29539967e+00 -2.18482999e+01
 -3.30740005e+00  4.61368001e+01 -8.18100052e-01  1.17095001e+01
  9.05850008e+00  7.53830050e+00 -2.62093998e

In [ ]:
def representar_review_como_vector_promedio(review):
    # Utiliza la función previamente definida para obtener los vectores suma y promedio
    vectores, promedio = representar_review_como_vector(review)
    return promedio


In [ ]:
# Buscar reviews que contienen la frase 'the only problem'
matching_reviews = train_df[train_df['review'].str.contains('the only problem', case=False)]

# Verificar si se encontraron reviews
if matching_reviews.empty:
    print("No se encontraron reviews que contengan la frase 'the only problem'.")
else:
    # Seleccionar la primera review que coincide
    review_ejemplo = matching_reviews['review'].values[0]

    # Obtener el vector promedio de la review
    promedio = representar_review_como_vector_promedio(review_ejemplo)

    # Encontrar las palabras más similares al vector promedio
    print("Palabras similares al vector promedio:")
    print(wordvectors.similar_by_vector(promedio, topn=10))


<ipython-input-21-907522245eb5>:7: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  vector = wordvectors.word_vec(palabra)


No está la palabra linklater's
No está la palabra $23
No está la palabra immerses
No está la palabra twentysomething
No está la palabra college-town
No está la palabra spouting
No está la palabra inadequacy
No está la palabra pervasiveness
No está la palabra they've
No está la palabra they'll
No está la palabra linklater
No está la palabra it's
No está la palabra linklater
No está la palabra meandering
No está la palabra pseudo-intellectuals
No está la palabra bribery-based
No está la palabra scooby-doo
No está la palabra unhinged
No está la palabra conspiracies
No está la palabra linklater
No está la palabra what's
No está la palabra aren't
No está la palabra strangeness
No está la palabra linklater
No está la palabra oddballs
No está la palabra well-deserved
No está la palabra can't
No está la palabra linklater
Palabras similares al vector promedio:
[('of', 0.7201033234596252), ('meI', 0.7176797389984131), ('the', 0.7163772583007812), ('that', 0.7091971039772034), ('to', 0.7056502699

# Red RNN  pero con Embeddings preentrenado

In [ ]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model.add(Conv1D(128, 5, activation="relu"))
    model.add(MaxPooling1D(5))
    model.add(Conv1D(128, 5, activation="relu"))
    model.add(MaxPooling1D(5))
    model.add(Conv1D(128, 5, activation="relu"))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=64*strategy.num_replicas_in_sync, validation_data=(xvalid_pad,yvalid_one_hot))


Epoch 1/20
19/19 [==============================] - 11s 163ms/step - loss: 0.7375 - accuracy: 0.4933 - val_loss: 0.6952 - val_accuracy: 0.4975
Epoch 2/20
19/19 [==============================] - 1s 61ms/step - loss: 0.6919 - accuracy: 0.5175 - val_loss: 0.6921 - val_accuracy: 0.5175
Epoch 3/20
19/19 [==============================] - 1s 61ms/step - loss: 0.6821 - accuracy: 0.5600 - val_loss: 0.6894 - val_accuracy: 0.5400
Epoch 4/20
19/19 [==============================] - 1s 61ms/step - loss: 0.6391 - accuracy: 0.6842 - val_loss: 0.6803 - val_accuracy: 0.5700
Epoch 5/20
19/19 [==============================] - 1s 62ms/step - loss: 0.5998 - accuracy: 0.6900 - val_loss: 0.7123 - val_accuracy: 0.5175
Epoch 6/20
19/19 [==============================] - 1s 62ms/step - loss: 0.4688 - accuracy: 0.8183 - val_loss: 0.6489 - val_accuracy: 0.5950
Epoch 7/20
19/19 [==============================] - 1s 64ms/step - loss: 0.2634 - accuracy: 0.9250 - val_loss: 0.7150 - val_accuracy: 0.6100
Epoch 8/20


In [ ]:
# Predecir en el conjunto de validación
preds = np.argmax(model.predict(xvalid_pad), axis=-1)

# Calcular la precisión
accuracy = metrics.accuracy_score(yvalid_encoded, preds)
print(f"Accuracy: {accuracy}")

# Generar el reporte de clasificación
target_names = label_encoder.classes_  # Esto debería ser ['negative', 'positive']
clas_report = classification_report(yvalid_encoded, preds, target_names=target_names)
print(clas_report)

13/13 [==============================] - 1s 11ms/step
Accuracy: 0.645
              precision    recall  f1-score   support

    negative       0.66      0.60      0.63       200
    positive       0.63      0.69      0.66       200

    accuracy                           0.65       400
   macro avg       0.65      0.65      0.64       400
weighted avg       0.65      0.65      0.64       400



# **LSTM**

In [ ]:
# Definir el modelo
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(2, activation='sigmoid'))  # Cambiado a 2 para clasificación binaria
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2700, 300)         11787000  
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 11947602 (45.58 MB)
Trainable params: 160602 (627.35 KB)
Non-trainable params: 11787000 (44.96 MB)
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=64*strategy.num_replicas_in_sync, validation_data=(xvalid_pad,yvalid_one_hot))

Epoch 1/20
19/19 [==============================] - 237s 12s/step - loss: 0.6932 - accuracy: 0.4900 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/20
19/19 [==============================] - 205s 11s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/20
19/19 [==============================] - 207s 11s/step - loss: 0.6932 - accuracy: 0.4983 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/20
19/19 [==============================] - 208s 11s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/20
19/19 [==============================] - 204s 11s/step - loss: 0.6932 - accuracy: 0.4792 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
19/19 [==============================] - 208s 11s/step - loss: 0.6932 - accuracy: 0.4642 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
19/19 [==============================] - 207s 11s/step - loss: 0.6933 - accuracy: 0.4800 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 

In [ ]:
# Predecir en el conjunto de validación
preds = np.argmax(model.predict(xvalid_pad), axis=-1)

# Calcular la precisión
accuracy = metrics.accuracy_score(yvalid_encoded, preds)
print(f"Accuracy: {accuracy}")

# Generar el reporte de clasificación
target_names = label_encoder.classes_  # Esto debería ser ['negative', 'positive']
clas_report = classification_report(yvalid_encoded, preds, target_names=target_names)
print(clas_report)

13/13 [==============================] - 11s 797ms/step
Accuracy: 0.5
              precision    recall  f1-score   support

    negative       0.50      1.00      0.67       200
    positive       0.00      0.00      0.00       200

    accuracy                           0.50       400
   macro avg       0.25      0.50      0.33       400
weighted avg       0.25      0.50      0.33       400



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **GRU**

In [ ]:
%%time
with strategy.scope():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(SpatialDropout1D(0.3))
    model.add(GRU(300))
    model.add(Dense(2, activation='sigmoid'))  # Cambiado a 2 para clasificación binaria
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2700, 300)         11787000  
                                                                 
 spatial_dropout1d_1 (Spati  (None, 2700, 300)         0         
 alDropout1D)                                                    
                                                                 
 gru_1 (GRU)                 (None, 300)               541800    
                                                                 
 dense_1 (Dense)             (None, 2)                 602       
                                                                 
Total params: 12329402 (47.03 MB)
Trainable params: 542402 (2.07 MB)
Non-trainable params: 11787000 (44.96 MB)
_________________________________________________________________
CPU times: user 323 ms, sys: 54.8 ms, total: 378 ms
Wall time: 372 ms


In [ ]:
model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=64*strategy.num_replicas_in_sync, validation_data=(xvalid_pad,yvalid_one_hot))

Epoch 1/20
19/19 [==============================] - 12s 373ms/step - loss: 0.6941 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/20
19/19 [==============================] - 6s 308ms/step - loss: 0.6933 - accuracy: 0.5033 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/20
19/19 [==============================] - 6s 309ms/step - loss: 0.6939 - accuracy: 0.4567 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/20
19/19 [==============================] - 6s 344ms/step - loss: 0.6932 - accuracy: 0.5067 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/20
19/19 [==============================] - 6s 317ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
19/19 [==============================] - 6s 316ms/step - loss: 0.6932 - accuracy: 0.4800 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
19/19 [==============================] - 7s 351ms/step - loss: 0.6932 - accuracy: 0.4933 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch

In [ ]:
# Predecir en el conjunto de validación
preds = np.argmax(model.predict(xvalid_pad), axis=-1)

# Calcular la precisión
accuracy = metrics.accuracy_score(yvalid_encoded, preds)
print(f"Accuracy: {accuracy}")

# Generar el reporte de clasificación
target_names = label_encoder.classes_  # Esto debería ser ['negative', 'positive']
clas_report = classification_report(yvalid_encoded, preds, target_names=target_names)
print(clas_report)

13/13 [==============================] - 1s 71ms/step
Accuracy: 0.5
              precision    recall  f1-score   support

    negative       0.50      1.00      0.67       200
    positive       0.00      0.00      0.00       200

    accuracy                           0.50       400
   macro avg       0.25      0.50      0.33       400
weighted avg       0.25      0.50      0.33       400



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **bidirectional LSTM**

In [ ]:

# Definir el modelo bidireccional LSTM
strategy = tf.distribute.MirroredStrategy()


with strategy.scope():
    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=max_len,
                        trainable=False))
    model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3)))
    model.add(Dense(2, activation='sigmoid'))  # Cambiado a 2 para clasificación binaria
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 2700, 300)         11883000  
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               439296    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 12322810 (47.01 MB)
Trainable params: 439810 (1.68 MB)
Non-trainable params: 11883000 (45.33 MB)
_________________________________________________________________


In [ ]:
# Definir el callback personalizado
import datetime
class TimeHistory(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_time_start = datetime.datetime.now()
        print(f"Epoch {epoch + 1} start time: {self.epoch_time_start}")

    def on_epoch_end(self, epoch, logs=None):
        self.epoch_time_end = datetime.datetime.now()
        print(f"Epoch {epoch + 1} end time: {self.epoch_time_end}")


EarlyStopping: Añadí el callback EarlyStopping para detener el entrenamiento si la validación no mejora después de 3 épocas, lo cual puede ahorrar tiempo.

In [ ]:
# Reduced batch size
batch_size = 64 * strategy.num_replicas_in_sync
# Crear una instancia de los callbacks
time_callback = TimeHistory()
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Entrenar el modelo con los callbacks
model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=batch_size, validation_data=(xvalid_pad, yvalid_one_hot), callbacks=[time_callback, early_stopping])

#model.fit(xtrain_pad, ytrain_one_hot, epochs=20, batch_size=batch_size, validation_data=(xvalid_pad, yvalid_one_hot))

Epoch 1 start time: 2024-07-03 17:13:04.251906
Epoch 1/20
19/19 [==============================] - 353s 19s/step - loss: 0.6798 - accuracy: 0.5633 - val_loss: 0.7000 - val_accuracy: 0.4900
Epoch 2 start time: 2024-07-03 17:18:56.870980
Epoch 2/20
19/19 [==============================] - 327s 17s/step - loss: 0.6704 - accuracy: 0.5825 - val_loss: 0.7071 - val_accuracy: 0.5100
Epoch 3 start time: 2024-07-03 17:24:24.024074
Epoch 3/20
19/19 [==============================] - 328s 17s/step - loss: 0.6556 - accuracy: 0.6250 - val_loss: 0.7141 - val_accuracy: 0.4800
Epoch 4 start time: 2024-07-03 17:29:51.689144
Epoch 4/20
19/19 [==============================] - 330s 17s/step - loss: 0.6308 - accuracy: 0.6600 - val_loss: 0.7334 - val_accuracy: 0.4750


In [ ]:
13:52
2:05
15 min por epoch

In [ ]:
# Predecir en el conjunto de validación
preds = np.argmax(model.predict(xvalid_pad), axis=-1)

# Calcular la precisión
accuracy = metrics.accuracy_score(yvalid_encoded, preds)
print(f"Accuracy: {accuracy}")

# Generar el reporte de clasificación
target_names = label_encoder.classes_  # Esto debería ser ['negative', 'positive']
clas_report = classification_report(yvalid_encoded, preds, target_names=target_names)
print(clas_report)

13/13 [==============================] - 20s 2s/step
Accuracy: 0.475
              precision    recall  f1-score   support

    negative       0.47      0.47      0.48       200
    positive       0.47      0.47      0.48       200

    accuracy                           0.48       400
   macro avg       0.47      0.47      0.48       400
weighted avg       0.47      0.47      0.47       400



# Resultados de Modelos de Clasificación de Reviews de Películas

| Modelo                       | Precisión Negativo | Recall Negativo | F1-Score Negativo | Precisión Positivo | Recall Positivo | F1-Score Positivo | Accuracy | Macro Avg F1-Score | Weighted Avg F1-Score |
|------------------------------|--------------------|-----------------|-------------------|--------------------|-----------------|-------------------|----------|--------------------|-----------------------|
| SimpleRNN                    | 0.49               | 0.69            | 0.57              | 0.48               | 0.29            | 0.37              | 0.49     | 0.47               | 0.47                  |
| CNN                          | 0.79               | 0.77            | 0.78              | 0.77               | 0.79            | 0.78              | 0.78     | 0.78               | 0.78                  |
| RNN con Embeddings Preentrenados | 0.66               | 0.60            | 0.63              | 0.63               | 0.69            | 0.66              | 0.65     | 0.64               | 0.64                  |
| LSTM                         | 0.50               | 1.00            | 0.67              | 0.00               | 0.00            | 0.00              | 0.50     | 0.33               | 0.33                  |
| GRU                          | 0.50               | 1.00            | 0.67              | 0.00               | 0.00            | 0.00              | 0.50     | 0.33               | 0.33                  |
| Bidirectional LSTM           | 0.47               | 0.47            | 0.48              | 0.47               | 0.47            | 0.48              | 0.48     | 0.48               | 0.47                  |

## Conclusiones

1. **Modelo CNN**:
   - **Mejor Rendimiento**: El modelo CNN tiene el mejor rendimiento en todos los aspectos y es el más recomendable para la tarea de clasificación de reviews de películas.

2. **Modelo RNN con Embeddings Preentrenados**:
   - **Rendimiento Medio**: Tiene un rendimiento intermedio, mejor que los modelos LSTM, GRU y Bidirectional LSTM pero inferior al modelo CNN.

3. **Modelos SimpleRNN, LSTM, GRU y Bidirectional LSTM**:
   - **Rendimiento Bajo**: Todos estos modelos tienen un rendimiento significativamente menor comparado con el modelo CNN, especialmente en la clase positiva.


Interesantes resultados:  Los modelos que utilizan embeddings preenrenados dieron resultados mas bajos y esto es  utilizamos embeddings en español y estos no son  adecuados para clasificar reviews en inglés. Para mejorar el rendimiento de los modelos que utilizan embeddings preentrenados, debería utilizar  embeddings entrenados en inglés.

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Supongamos que `model` es el modelo CNN ya entrenado
# y `tokenizer` es el tokenizer ajustado en el conjunto de datos de entrenamiento

def preprocesar_review(review, tokenizer, max_len):
    # Convertir la review en una secuencia de índices
    seq = tokenizer.texts_to_sequences([review])
    # Aplicar padding a la secuencia
    padded_seq = pad_sequences(seq, maxlen=max_len)
    return padded_seq

def clasificar_review(review, model, tokenizer, max_len):
    # Preprocesar la review
    preprocessed_review = preprocesar_review(review, tokenizer, max_len)
    # Hacer la predicción
    pred = model.predict(preprocessed_review)
    # Obtener la clase predicha
    pred_class = np.argmax(pred, axis=1)
    return 'positive' if pred_class[0] == 1 else 'negative'

# Ejemplo de uso:
nueva_review = "This movie was incredible, the acting was spectacular and the plot was very involving."
resultado = clasificar_review(nueva_review, model, tokenizer, max_len)
print(f"La review es: {resultado}")


1/1 [==============================] - 4s 4s/step
La review es: positive
